# Assignment 3: Language Models: Auto-Complete

In this assignment, you will build an auto-complete system.  Auto-complete system is something you may see every day
- When you google something, you often have suggestions to help you complete your search. 
- When you are writing an email, you get suggestions telling you possible endings to your sentence.  

By the end of this assignment, you will develop a prototype of such a system.

<img src = "./images/stanford.png" style="width:700px;height:300px;"/>

## Table of Contents
- [1 - Load and Preprocess Data](#1)
    - [1.1 - Load the Data](#1.1)
    - [1.2 - Pre-process the Data](#1.2)
        - [Exercise 1- split_to_sentences (UNQ_C1)](#ex-1)
        - [Exercise 2 - tokenize_sentences (UNQ_C2)](#ex-2)
        - [Exercise 3 - get_tokenized_data (UNQ_C3)](#ex-3)
        - [Exercise 4 - count_words (UNQ_C4)](#ex-4)
        - [Exercise 5 - get_words_with_nplus_frequency (UNQ_C5)](#ex-5)
        - [Exercise 6 - replace_oov_words_by_unk (UNQ_C6)](#ex-6)
        - [Exercise 7 - preprocess_data (UNQ_C7)](#ex-7)
- [2 - Develop n-gram based Language Models](#2)
    - [Exercise 8 - count_n_grams (UNQ_C8)](#ex-8)
    - [Exercise 9 - estimate_probability (UNQ_C9)](#ex-9)    
- [3 - Perplexity](#3)
    - [Exercise 10 - calculate_perplexity (UNQ_C10)](#ex-10)
- [4 - Build an Auto-complete System](#4)
    - [Exercise 11 - suggest_a_word (UNQ_C11)](#ex-11)

A key building block for an auto-complete system is a language model.
A language model assigns the probability to a sequence of words, in a way that more "likely" sequences receive higher scores.  For example, 
>"I have a pen" 
is expected to have a higher probability than 
>"I am a pen"
since the first one seems to be a more natural sentence in the real world.

You can take advantage of this probability calculation to develop an auto-complete system.  
Suppose the user typed 
>"I eat scrambled"
Then you can find a word `x`  such that "I eat scrambled x" receives the highest probability.  If x = "eggs", the sentence would be
>"I eat scrambled eggs"

While a variety of language models have been developed, this assignment uses **N-grams**, a simple but powerful method for language modeling.
- N-grams are also used in machine translation and speech recognition. 


Here are the steps of this assignment:

1. Load and preprocess data
    - Load and tokenize data.
    - Split the sentences into train and test sets.
    - Replace words with a low frequency by an unknown marker `<unk>`.
1. Develop N-gram based language models
    - Compute the count of n-grams from a given data set.
    - Estimate the conditional probability of a next word with k-smoothing.
1. Evaluate the N-gram models by computing the perplexity score.
1. Use your own model to suggest an upcoming word given your sentence. 

In [2]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

import w3_unittest
nltk.data.path.append('.')

[nltk_data] Downloading package punkt to /Users/anrui/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<a name='1'></a>
## 1 - Load and Preprocess Data

<a name='1.1'></a>
### 1.1 - Load the Data
You will use twitter data.
Load the data and view the first few sentences by running the next cell.

Notice that data is a long string that contains many many tweets.
Observe that there is a line break "\n" between tweets.

In [3]:
with open("./data/en_US.twitter.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")


Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


<a name='1.2'></a>
### 1.2 - Pre-process the Data

Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
1. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
1. Assign sentences into train or test sets.
1. Find tokens that appear at least N times in the training data.
1. Replace tokens that appear less than N times by `<unk>`


Note: we omit validation data in this exercise.
- In real applications, we should hold a part of data as a validation set and use it to tune our training.
- We skip this process for simplicity.

<a name='ex-1'></a>
### Exercise 1- split_to_sentences

Split data into sentences.

In [ ]:
# UNIT TEST COMMENT: Candidate for Table Driven Tests 
### UNQ_C1 GRADED_FUNCTION: split_to_sentences ###
def split_to_sentences(data):
    """
    Split data by linebreak "\n"
    
    Args:
        data: str
    
    Returns:
        A list of sentences
    """
    ### START CODE HERE ###
    sentences = None
    ### END CODE HERE ###
    
    # Additional clearning (This part is already implemented)
    # - Remove leading and trailing spaces from each sentence
    # - Drop sentences if they are empty strings.
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    
    return sentences    